# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:

#pip install gmaps
city_df=pd.read_csv("citydatainfo.csv")
city_df.head()

,Unnamed: 0,City,Country,Date,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,whyalla,AU,1619910581,137.5833,-33.0333,59.00,82,1,6.91
1,1,thompson,CA,1619910581,-97.8558,55.7435,23.00,86,90,9.22
2,2,vanimo,PG,1619910534,141.3028,-2.6741,80.98,80,71,4.61
3,3,jamestown,US,1619910581,-79.2353,42.0970,56.80,35,90,20.71
4,4,arraial do cabo,BR,1619910431,-42.0278,-22.9661,72.90,68,100,11.52


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=g_key)


In [19]:
location=city_df[["Latitude","Longitude"]]
weight=city_df["Humidity"]
fig=gmaps.figure()
heatlayer=gmaps.heatmap_layer(location,weights=weight,dissipating=False,max_intensity=400,point_radius=5)
fig.add_layer(heatlayer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
perfect_weather=city_df.loc[(city_df["Temperature"] >= 70) 
                        &(city_df["Temperature"] <= 80)
                        &(city_df["Wind Speed"] <= 10)
                        &(city_df["Cloudiness"] == 0)].dropna()

perfect_weather.head()


# hotel_df= perfect_weather

# hotel_df.insert(3,"Hotel","")

# hotel_df.head()

,Unnamed: 0,City,Country,Date,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed
29,29,saint-philippe,RE,1619910455,55.7679,-21.3585,73.63,73,0,5.75
133,133,pindorama,BR,1619910616,-48.9072,-21.1858,78.80,41,0,8.05
197,197,saint-joseph,RE,1619910633,55.6167,-21.3667,73.71,68,0,6.91
224,224,pozo colorado,PY,1619910447,-58.7972,-23.4923,74.84,46,0,6.02
274,274,saint-pierre,RE,1619910075,55.4781,-21.3393,73.69,73,0,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df= perfect_weather[["City","Country","Longitude","Latitude"]].copy()

hotel_df.insert(4,"Hotel Name","")

hotel_df.head()

,City,Country,Longitude,Latitude,Hotel Name
29,saint-philippe,RE,55.7679,-21.3585,
133,pindorama,BR,-48.9072,-21.1858,
197,saint-joseph,RE,55.6167,-21.3667,
224,pozo colorado,PY,-58.7972,-23.4923,
274,saint-pierre,RE,55.4781,-21.3393,


In [22]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, hotel in hotel_df.iterrows():
    lat=hotel["Latitude"]
    long=hotel["Longitude"]
    params["location"]=f"{lat},{long}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response= requests.get(base_url,params=params)
    url_lists=response.json()
    try:
        hotel_df.loc[index, "Hotel Name"]=url_lists["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/results...skipping")
hotel_df       

,City,Country,Longitude,Latitude,Hotel Name
29,saint-philippe,RE,55.7679,-21.3585,"Chambres d'hôte ""La Trinité"""
133,pindorama,BR,-48.9072,-21.1858,PINDORAMA GOLDEN PLAZA HOTEL
197,saint-joseph,RE,55.6167,-21.3667,"""Plantation Bed and Breakfast"
224,pozo colorado,PY,-58.7972,-23.4923,Hospedaje la Amistad
274,saint-pierre,RE,55.4781,-21.3393,Lindsey Hôtel
325,rafaela,AR,-61.4867,-31.2503,Parra Hotel & Suites
344,riyadh,SA,46.7219,24.6877,"InterContinental Riyadh, an IHG Hotel"
390,itoman,JP,127.6694,26.1247,Southern Beach Hotel & Resort Okinawa


In [23]:
print(url_lists)

{'html_attributions': [], 'next_page_token': 'ATtYBwKNl2_Sw9-P4U7_vIzs2Vx3m-QOVQ4-Oh4y-Wd1KgwWrB_7aLMkR4VWpUht00ty-S_bazt11EH8XAa41w1vDHQGFyelb6Sq6HuJpcw4VRVLExyhjx7wCZreGZs7rre8lWhRoa8FH3najJW5bhuPvJtkkuUu15alfc7Afh4JIRz9Ww6G2BU1mUK99XZ_hktSS56SXOp0oPQrKWHkdTDxm-2Q0TkcDWGFyQ-Biru0HqYgSnOuqd5DGlzK0DeJf8tgBn_WKBr22Ink3-XAhs9PsxhYHMcevXxozxQezuaUaHy3kMG16GjcDK-lgMKAOf5v_vzIXIeBkAv9XgtdU5QQLP827U4PtKDxqbvqHAxhBSXg5Yx3l5-staXjB2kbPHKgLKOx99dj_HIUC-bqZtGq-dvKBUjWcrPLIvE6B22-L5RU0I2E6v4FuCHw', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 26.13202729999999, 'lng': 127.6529371}, 'viewport': {'northeast': {'lat': 26.13348968029149, 'lng': 127.6544053302915}, 'southwest': {'lat': 26.13079171970849, 'lng': 127.6517073697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Southern Beach Hotel & Resort Okinawa', 'photos': [{'height': 1282, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111708

In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_name = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]


In [25]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_name)
fig=gmaps.figure()
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))